In [1]:
import pandas as pd
import numpy as np
import pyunpack
import math
import json

from data_formatters.m5 import M5Formatter
from data_formatters.base import DataTypes, InputTypes

from pytorch_dataset import TFTDataset
from models import GatedLinearUnit
from models import GateAddNormNetwork
from models import GatedResidualNetwork 
from models import ScaledDotProductAttention
from models import InterpretableMultiHeadAttention
from models import VariableSelectionNetwork

from quantile_loss import QuantileLossCalculator
from quantile_loss import NormalizedQuantileLossCalculator

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from argparse import ArgumentParser

import matplotlib.pyplot as plt

## Creating Datasets

In [2]:
m5_data = pd.read_pickle('/home/daniel/github/kaggle_competitions/M5_Forecasting_Accuracy/data/full_data.pkl')

In [9]:
m5_data.shape

(46881677, 34)

In [8]:
m5_data.groupby(['id']).apply(lambda x: len(x)).value_counts().to_frame()

,0
1941,10932
1934,1043
1927,544
1906,301
1920,280
...,...
191,1
177,1
135,1
289,1


In [3]:
data_formatter = M5Formatter()
train, valid, test = data_formatter.split_data(m5_data)

Formatting train-valid-test splits.
Setting scalers with training data...
1913
1884
1884
1884
1884
1884
1884
1877
1884
1884
1884
1884
1884
1884
1884
1877
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1485
1485
1485
1485
1485
1485
1485
1485
1485
1485
1884
1884
1884
1884
1884
1884
1884
1884
1884
1863
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1625
1520
1331
1625
1604
1506
1457
1618
1597
1534
1485
1485
1485
1485
1478
1471
1478
1478
1478
1471
1401
1702
1702
1702
1793
1758
1793
1366
1366
1072
1884
1821
1884
1884
1884
1884
1870
1821
1884
1884
1702
1702
1702
1702
1793
1793
1793
1702
1702
1702
1884
1884
1884
1485
1877
1884
1884
1884
1884
1884
1044
1044
1044
750
1359
1366
1359
1345
1338
1345
1870
1814
1807
1814
1807
1814
1793
1814
1821
1814
1884
1884
1884
1807
1884
1793
1884
1884
1884
1884
1478
1485
1478
1478
1478
1478
1464
1478
1478
1471
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
18

1478
1478
1121
1457
1471
1471
1471
1485
1478
1884
1877
1863
1884
1877
1884
1877
1884
1856
1870
666
764
764
757
764
666
645
666
764
666
1492
1485
1478
1478
1485
1492
1485
1478
1485
1492
1135
1135
1135
1135
1128
1135
1128
1135
1121
1135
1702
1702
1702
1702
1765
1793
1793
1366
1373
1366
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1002
1002
1401
1002
1401
1793
1793
1002
1002
1002
1884
1884
1884
1884
1884
1884
1884
1884
1884
1870
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1877
1884
1884
1863
1884
1884
1884
1814
1884
1884
1877
1884
1884
1877
1884
1884
1884
1884
1884
1884
1877
1884
1884
1877
1702
1702
1702
1702
1793
1758
1765
1702
1373
1702
666
659
652
652
638
596
638
673
652
673
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1135
1135
1135
1135
1121
1128
1114
1135
1135
1128
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1814
1821
1814
1779
1821
1807
1807
1821
1821
1814
1478
771
1485
1478
1478
1471
1464
1471
1485
1478
1884
1884
1800
1884
1884
1

1884
1884
1884
1247
1429
1884
1135
309
1044
1044
1128
1135
1128
1044
1128
1044
1849
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1849
1884
1884
1884
302
1884
1884
1884
1884
1884
1177
1177
1884
736
309
722
722
743
715
470
729
477
687
1884
309
1884
1884
1884
1884
1884
1254
1422
1884
1884
309
1884
1884
1884
1884
1884
1240
1422
1884
708
302
708
323
708
708
708
708
708
701
736
715
750
743
736
617
729
743
743
603
1884
260
1877
1884
1870
1884
1884
1254
1422
1884
1835
302
1835
1807
1821
1842
1436
1254
1422
1821
1436
309
1429
1436
211
1240
204
1254
1422
1429
1884
302
1884
1632
1884
1884
1632
1247
1359
1884
1884
1884
1884
1884
1884
1884
1884
1842
1884
1884
1030
309
1023
1016
1016
1023
771
1023
988
1016
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
309
1884
1884
1884
1884
1212
1240
1422
1884
673
302
687
673
673
645
638
659
659
680
1534
1534
1534
1534
1534
1534
1534
1534
1534
1534
1779
1765
1779
1772
1772
1772
1730
1772
1758
1772
1884
1884
1884
1884
1884

1709
1716
1709
1716
1114
1422
1716
1772
302
1856
1870
1723
1821
1079
1254
1086
1863
1590
330
1632
1590
1583
1583
1184
1254
1422
1590
1359
218
1485
1485
1478
1485
1485
1184
1359
1485
1408
1401
1401
1408
1401
1401
1408
1408
1408
1408
1877
1877
1877
1877
1877
1877
1877
1254
1877
1877
1884
1590
1884
1884
1877
1884
1877
1254
1751
1884
743
302
743
666
750
736
470
750
323
750
1884
309
1884
1884
1884
1884
1884
1247
1422
1884
1884
302
1814
1884
1877
1884
1884
1247
1429
1884
1884
1184
1884
1884
1884
1884
1884
1254
1359
1884
897
295
890
785
897
890
785
890
897
897
617
309
617
554
617
554
617
617
617
617
1884
309
1884
1884
1884
1884
1884
1485
1492
1884
1884
309
1884
1779
1884
1877
1884
1240
1429
1639
1429
1429
1429
1429
1429
1429
1429
1254
1429
1429
1023
309
1023
1023
1023
1023
974
925
1023
974
1884
1884
1884
1877
1884
1884
1884
1247
1737
1884
1765
680
1765
1758
1765
1765
1541
1114
1422
1765
1884
309
1884
1884
1884
1884
1884
1254
1422
1884
1009
302
1023
1023
967
1023
1002
1023
995
1002
764
183
764

421
435
421
421
1520
1520
1520
1513
1478
1478
1478
1268
1394
1450
1205
1198
1205
1205
1205
1205
1205
1198
1198
1198
799
799
792
792
799
799
792
792
799
799
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1814
1884
603
547
603
533
540
533
540
603
533
533
1821
1821
1821
1814
1821
1821
1870
1254
1226
1751
1884
1884
1884
1884
1884
1884
1884
1254
1884
1884
1884
1884
1884
1884
1884
1884
1877
1254
1884
1884
1023
1023
1023
1023
1016
1023
1023
1002
1016
1016
1569
1569
1569
1569
1569
1562
1569
1569
1569
1562
1688
1681
1688
1674
1688
1688
1688
1681
1681
1695
1842
1884
1842
1884
1842
1849
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1877
1884
1884
414
407
414
421
428
428
428
1646
1646
1639
575
981
652
554
1765
1681
589
666
400
666
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
764
729
778
764
750
771
736
764
757
778
1541
1380
1541
1408
1541
1541
1541
1380
1429
1394
1884
1884
1884
1688
1884
1884
1884
1821
1884
379
1520
1520
1555
1520
1562
1625


...

1800
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1492
1492
1492
1492
1492
1492
1492
1492
1492
1492
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
456
617
617
533
596
358
456
512
596
624
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1849
1548
1884
1737
1884
1884
1884
1884
1884
1884
1884
1842
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1870
1877
1870
1884
1884
1884
1884
1877
1884
687
687
694
666
687
589
589
596
694
554
1884
1884
1884
1877
1884
1884
1884
1821
1884
1884
953
953
953
953
953
953
953
953
946
953
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
610
603
610
610
610
589
610
610
617
540
1135
1135
1135
1128
1135
1128
1114
1079
1128
1114
813
820
813
813
820
813
715
806
806
806
1037
1030
1037
1030
1037
1044
1037
1030
1030
1023
1415
1331
1156
1415
1422
1422
1422
43
1128
1422
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
533
519
533
533
526
526
540
533
540
519
1884
18

1464
1653
1653
1646
1317
1646
1653
1646
1653
1639
1639
1758
1751
1758
1737
1744
1751
1758
1751
1737
1611
1884
1884
1884
1884
1877
1884
1884
1884
1870
1884
1877
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1877
1884
1807
1814
1884
1884
589
589
526
589
589
561
589
575
526
365
1884
1884
1884
1877
1842
1863
1870
1863
1870
1884
1884
1884
1884
1877
1884
1884
1884
1884
1786
1884
862
848
862
855
848
848
855
855
848
834
1877
1884
1884
1821
1877
1884
1884
1884
1884
1884
1814
1821
1814
1814
1814
1814
1814
1814
1814
1814
1072
1086
1086
1065
1443
1436
1086
1023
1079
1009
1170
1170
1163
1163
1163
1142
1163
1170
1163
1163
1884
1877
1884
1877
1884
1884
1877
1884
1758
1877
1646
1702
1709
1716
1646
1723
1709
1709
1527
1681
995
1023
1044
953
1058
1058
1002
855
911
1058
1877
1884
1884
1884
1884
1884
1884
1877
1884
1884
1877
1884
1884
1884
1884
1884
1877
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1198
1198
1184
1163
1184
1184
1184
1184
1198
1184
1240
1247
1247
1226

1863
1884
1877
1884
1128
1128
1128
1128
407
407
407
1128
1107
1128
1884
1884
1884
1884
1884
1842
1884
1884
1884
1884
1884
1884
1884
1884
1877
1884
1884
1884
1863
1884
1807
1814
1814
1653
1800
1814
1807
1814
1814
1807
1884
1884
1884
1884
1884
1884
1884
1884
1744
1737
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1786
1779
1800
1807
1583
1800
1884
1765
1520
1877
1002
1093
1093
1072
1086
1100
1100
1086
1100
1065
1877
1884
1632
1744
1849
1870
1639
1863
1723
1863
1373
1366
1373
1352
1359
1373
1366
1366
1352
1373
1884
1877
1884
1548
1863
1884
1877
1884
1884
1870
1884
1884
1884
1884
1877
1884
1884
1877
1884
1884
1800
1800
1793
1800
1800
1793
1751
1807
1786
1793
1884
1884
1884
1884
1884
1884
1884
1884
1884
1772
1800
1884
1884
1800
1800
1884
1884
1807
1709
1884
1884
1884
1884
1863
1856
1870
1877
1884
1877
1863
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
862
904
904
897
897
897
904
890
897
897
1884
1870
1884
1884
1884
1877
1884
1877
1884
1884
1884
1884
1884
1884
1870
1877
1884
1877
188

1884
1884
1884
1877
1884
1877
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
855
848
855
841
575
834
848
848
792
848
533
631
533
624
533
533
533
533
533
533
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1821
1884
1877
1884
1884
1884
1884
1877
1814
1870
1639
1877
1870
1870
1870
1849
1856
1177
1177
1177
1170
1156
1170
1170
1156
1163
1170
1142
1142
1135
1128
1142
1114
1142
1114
1128
1142
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
610
610
610
603
603
610
610
603
610
603
1884
1884
1884
1856
1884
1884
1884
1877
1884
1884
1884
1884
1884
1884
1877
1884
1884
1884
1884
1884
1618
1548
1674
1576
1618
1667
1688
1667
1695
1681
1856
1884
1877
1863
1870
1877
1877
1884
1870
1877
1884
1884
1884
1884
1884
1884
1884
1884
1884
1884
1863
1884
1884
1870
1877
1884
1877
1884
1751
1884
757
631
778
645
757
806
806
729
701
414
1114
1100
918
904
1100
911
1107
911
904
1093
1884
1884
1884
1884
1863
1863
1884
1884
1884
1884
666
6

ValueError: No objects to concatenate